## Die erste eigene ROS Node

In dieser Übung erzeugen wir eine erste [**ROS Node**](http://wiki.ros.org/Nodes) und verbinden sie mit dem ROS Netzwerk.
Dazu müssen wir eine Node erzeugen und sie beim Verteiler, dem [**roscore**](http://wiki.ros.org/roscore), registrieren.
Prinzipiell können wir jede beliebige IP Adresse angeben, unter dem der roscore erreichbar ist. Hier benutzen wir den lokalen roscore Server, sodass wir die Standardeinstellungen nutzen können.


Wir werden den Großteil des Codes in [Python 2.7](https://docs.python.org/2.7/) nutzen und dabei auf die [Python Bibliotheken von ROS](http://wiki.ros.org/rospy) zurückgreifen.

Mit Python zu Programmieren ist relativ einfach und erfordert keinen großen Prozess um den Code ausführbar zu machen, was uns entgegen kommt

Zunächst importieren wir einige wichtige Bilbiotheken die wir im Verlauf benötigen werden.
Eine Zelle (Markiert durch ein **In[ ]** davor) kann mit der Tastenkombination Shift + Enter ausgeführt, wenn ihr sie markiert habt. Wenn sie erfolgreich ausgeführt wurde, erscheint eine Zahl in den Klammern, die eine fortlaufende Nummer in der Ausführung darstellt.

Nachdem ihr die Zelle ausgeführt habt, sollte davor ein **In [1]** erscheinen

In [2]:
import rospy
import roslaunch
import rosnode
import os
import subprocess

Wir starten nun den Verteiler für die Kommunikation, den **roscore**

In [3]:
roscore_process=subprocess.Popen(["roscore"])

Wir können nun die bereits existierenden **Nodes** anzeigen lassen. Mit dem Befehl *"get_node_names()"* können wir alle Nodes auflisten:

In [4]:
rosnode.get_node_names()

['/rosout']

Neue Nodes lassen sich durch den Befehl *.init_node(name)* erzeugen. Gebt euren eigenen Node Namen an!

In [5]:
node_name = "Meine_Lieblingsnode";
rospy.init_node(node_name, anonymous=True)

Wir können jetzt sehen, dass die neue Node erzeugt wurde. Lasst euch nochmal die Namen aller Nodes anzeigen! 

In [6]:
# Hier den code einfügen, um die Namen aller Nodes Anzeigen zu lassen!

['/rosout', '/Meine_Lieblingsnode_5719_1548412462341']

## Kommunizieren in ROS: Topics, Services, Parameter

Für die Kommunikation der verschiedenen ROS Nodes untereinander stellt das ROS Framework für verschiedene Zwecke entsprechende Möglichkeiten zur Verfügung.

Kurz zusammengefasst:
**Topics** sind einfache Kommunikationskanäle, in die jede Node schreiben kann, solange das *Format* korrekt ist. Jede Node kann auch ohne Verzögerung daraus lesen.

Ein typisches **Topic** wäre zum Beispiel die Information über die Temperatur. Der Temperatursensor publiziert hierbei als *Publisher* die aktuelle Temperatur in regelmäßigen Abständen. Es ist hier für den Temperatursensor unwichtig, Rückmeldung zu bekommen, was mit den Daten geschieht oder was daraus berechnet wird. Verschiedene andere Nodes, die z.B. den Temperaturverlauf visualisieren oder eine Glatteiswarnung herausgeben, lesen die Information, die auf dem Topic veröffentlich wird. Man nennt sie *Subscriber*

**Services** sind hingegen bilaterale Kommunikationskanäle, d.h. eine Node stellt eine Anfrage und wartet auf eine Antwort von einer anderen Node. 

Ein typischer **Service** wird zum Beispiel von einer Bildverarbeitungsnode bereitgestellt. Soll auf einem Bild bespielsweise ein Objekt detektiert werden, so wird der Bildverarbeitungsnode ein Bild per *service call* zur verfügung gestellt. Diese führt nun den Detektionsalgorithmus aus und liefert in der Antwort die Position des Objektes zurück.

**Parameter** sind dagegen eine gemeinsame Datenbank von Werten, die von vielen *Nodes* benötigt werden. Üblicherweise werden hier nicht- bzw. nur spordische veränderliche Werte bereitgestellt, die nur selten verarbeitet werden, wie z.B. die Hardwarekonfiguration der Roboter.

## Der erste Roboter: *TurtleBot*

Beginnen wir mit dem ersten Roboter, den man über Simulation (*TurtleSim*), aber auch potenziell als reale Hardware über ROS steuern kann - dem *TurtleBot*

![](img/TurtleBot2.jpg)


In [7]:
# Einfach diese Zelle ausführen um die TurtleSim Simulation zu starten!

launch = roslaunch.scriptapi.ROSLaunch()
launch.start()
node1=roslaunch.core.Node("turtlesim", "turtlesim_node",name="turtlesim_node")
launch.launch(node1)

started roslaunch server http://AURORA-ML:36159/

SUMMARY

PARAMETERS
 * /rosdistro: kinetic
 * /rosversion: 1.12.14

NODES

ROS_MASTER_URI=http://localhost:11311
process[turtlesim_node-1]: started with pid [18092]


Cool! Ihr solltet jetzt einen kleinen Schildkrötenroboter in einem seperaten Fenster sehen!

Die Simulation hat bereits ein Inferface zu ROS, es ist also leicht möglich, den Roboter Befehle über die ROS Kommunikationskanäle zu geben.

Lasst uns mal schauen, wie wir mit dem Roboter kommunizieren können!

### ROS Topic

Fangen wir mal mit einem **Topic** an. Mit **!rostopic** lassen sich alle Kommunikationskanäle anzeigen, die registriert sind.


Lasst uns zunächst einmal sehen, welche Topics überhaupt von TurtleSim zur Verfügung gestellt werden

In [10]:
!rostopic list

/rosout
/rosout_agg
/turtle1/cmd_vel
/turtle1/color_sensor
/turtle1/pose


Aha! Die Node "turtle1" stellt 3 Kanäle zur Verfügung
* color_sensor - was irrelevant ist für uns zunächst
* pose - bei dem die aktuelle Position publiziert wird
* cmd_vel - bei der man die Geschwindigkeit senden kann

Jedes Topic hat dabei ein bestimmtes Format - dieses kann man mit

*!rostopic info topic_name* 

anzeigen lassen:

In [36]:
!rostopic info /turtle1/pose

Type: turtlesim/Pose

Publishers: 
 * /turtlesim_node (http://AURORA-ML:35015/)

Subscribers: None




Hier sehen wir, dass 
* Das Topic vom Typ *turtlesim/Pose* ist
* Dass eine Node in das Topic schreibt
* Dass niemand auf dem Topic regelmäßig liest (*Subscribers*)

Wir können jetzt einmal lesen, was für Informationen dort veröffentlicht werden.
Dazu müssen wir die Formatdefinition (als ein *.msg* File bereits vordefiniert) angeben und das Topic.

Dazu nutzen wir die Funktion *wait_for_message*

In [44]:
from turtlesim.msg import Pose

rospy.wait_for_message("/turtle1/pose", Pose, timeout=1) 

x: 5.544444561
y: 5.544444561
theta: 0.0
linear_velocity: 0.0
angular_velocity: 0.0

Um schnell nachzusehen, welche Informationen eine beliebige Funktion benötigt, kann man übrigens einfach ein ? davor oder dahinter stellen:

In [46]:
rospy.wait_for_message?

In ROS sind bereits verschiedene Formatdefinitionen (*messages*) vordefiniert. Man kann sie sich leicht auflisten lassen und genauere Informationen darüber abrufen:

In [47]:
!rosmsg list

actionlib/TestAction
actionlib/TestActionFeedback
actionlib/TestActionGoal
actionlib/TestActionResult
actionlib/TestFeedback
actionlib/TestGoal
actionlib/TestRequestAction
actionlib/TestRequestActionFeedback
actionlib/TestRequestActionGoal
actionlib/TestRequestActionResult
actionlib/TestRequestFeedback
actionlib/TestRequestGoal
actionlib/TestRequestResult
actionlib/TestResult
actionlib/TwoIntsAction
actionlib/TwoIntsActionFeedback
actionlib/TwoIntsActionGoal
actionlib/TwoIntsActionResult
actionlib/TwoIntsFeedback
actionlib/TwoIntsGoal
actionlib/TwoIntsResult
actionlib_msgs/GoalID
actionlib_msgs/GoalStatus
actionlib_msgs/GoalStatusArray
actionlib_tutorials/AveragingAction
actionlib_tutorials/AveragingActionFeedback
actionlib_tutorials/AveragingActionGoal
actionlib_tutorials/AveragingActionResult
actionlib_tutorials/AveragingFeedback
actionlib_tutorials/AveragingGoal
actionlib_tutorials/AveragingResult
actionlib_tutorials/FibonacciAction
actionlib_tutorials/FibonacciActionFeedback
action

In [49]:
!rosmsg info turtlesim/Pose

float32 x
float32 y
float32 theta
float32 linear_velocity
float32 angular_velocity



### Schildkrötenbefehle

Wir können jetzt einfach mal versuchen, der Schildkröte einen Befehl zu schicken.
Das könne wir auf dem Kanal /turtle1/cmd_vel.
Findet doch mal heraus, welcher Message-Typ hier genutzt wird und wie wir ihn importieren können - das geht genau so wie gerade eben!

In [50]:
# implement here!

Type: geometry_msgs/Twist

Publishers: None

Subscribers: 
 * /turtlesim_node (http://AURORA-ML:35015/)




In [8]:
# we import the definition of a Twist message
from geometry_msgs.msg import Twist 

Wir erzeugen nun erst einmal einen *Publisher*, der uns ein einfaches und persistentes Interface bietet, um *Messages* zu publizieren. Wir nutzen dazu die Publisher Klasse und übergeben noch die Information zu Kanalnamen und dem zugehörigem Messageformat:

In [9]:
# we create a publisher for a "/turtle1/cmd_vel" topic
twist_publisher = rospy.Publisher("/turtle1/cmd_vel",Twist) 

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SyntaxWarning: The publisher should be created with an explicit keyword argument 'queue_size'. Please see http://wiki.ros.org/rospy/Overview/Publishers%20and%20Subscribers for more information.
  


Um jetzt eine Message zu schicken, holen wir uns das Format als Blaupause und schauen uns mal an, welche Information dabei geschickt wird:

In [10]:
message = Twist()

In [11]:
message

linear: 
  x: 0.0
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 0.0

Ändern wir also die Sollgeschwindigkeit einmal für die Drehung um die Z-Achse und senden diese mit dem *publish* Befehl zu unserer Schildkröte

In [12]:
message.angular.z = 1

In [13]:
message

linear: 
  x: 0.0
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 1

In [14]:
twist_publisher.publish(message)

Wie wir sehen, führt die Schildkröte den Befehl nur für eine gewisse Zeit aus und resettet den Befehl danach wieder. Wir müssen also dauerhaft einen Befehl senden, wenn wir wollen, dass die Schildkröte ihn länger ausführt.

Um uns das Leben einfacher zu machen, definieren wir eine Funktion, die uns die repetetive Ausführung übernimmt:

In [15]:
def move_turtle(forward_vel=0):
    '''A function to move turtle from turtlesim simulation
    
    Args:
        forward_vel (float): Linear velocity
        rotation_vel (float): Angular velocity'''
    message=Twist()
    
    message.linear.x=forward_vel
    
    twist_publisher.publish(message)
    

In [16]:
move_turtle(1)

Leider funkioniert das Drehen um die Z Achse noch nicht. Fügt doch das bitte zur Funktion hinzu!

In [17]:
def move_turtle(forward_vel=0,rotation_vel=0):
    '''A function to move turtle from turtlesim simulation
    
    Args:
        forward_vel (float): Linear velocity
        rotation_vel (float): Angular velocity'''
    message=Twist()
    
    message.linear.x=forward_vel
    
    twist_publisher.publish(message)

Das ganze lässt sich jetzt noch in einen schönen Slider verpacken, der automatisch bei Änderungen den Befehl publiziert und man kann ein bisschen damit Herumspielen:

In [18]:
import ipywidgets
ipywidgets.interact(move_turtle,
                    forward_vel=ipywidgets.FloatSlider(min=-10,max=10,step=2,value=0),
                   rotation_vel= ipywidgets.FloatSlider(min=-3,max=3,value=0));

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPTAuMCwgZGVzY3JpcHRpb249dSdmb3J3YXJkX3ZlbCcsIG1heD0xMC4wLCBtaW49LTEwLjAsIHN0ZXA9Mi4wKSwgRmxvYXTigKY=


### Unsere erste Subscription!

Ein **Subscriber** verarbeitet Messages, die auf einem bestimmten **Topic** publiziert werden. Es übergibt diese an eine Funktion, die die Message verarbeitet.

Das einfachste ist erst einmal, die Message einfach auszugeben.
Nachfolgend ist bereits der ein Subscriber definiert, es fehlen nur noch die Informationen zum Topic Kanal und zur Messageformat, die ihr noch hinzufügen müsst.

Subscribed einfach mal auf dem Kanal, auf dem die Sollgeschwindigkeit publiziert wird!

In [122]:
topic_name = "/turtle1/cmd_vel"
msg_type = Twist

def write_reversed(message):
    print(message)
    
# now we create a new subscriber
new_subscriber = rospy.Subscriber(topic_name,msg_type,write_reversed)

Jetzt können wir uns auch noch anschauen, was passiert, wenn wir eine neue Message publizeren:

In [125]:
move_turtle(1,1)

linear: 
  x: 1.0
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 1.0


Das wird aber auch schnell unübersichtlich: Wenn man zum Beispiel viele Messages publiziert, wie z.B. mit dem Slider. Ihr könnt das ja mal ausprobieren. Um die Verarbeitung der Messages wieder aufzuheben kann man auch die Weiterleitung einfach aufheben mittels der *unregister* Funktion:

In [126]:
import ipywidgets
ipywidgets.interact(move_turtle,
                    forward_vel=ipywidgets.FloatSlider(min=-10,max=10,step=2,value=0),
                   rotation_vel= ipywidgets.FloatSlider(min=-3,max=3,value=0));

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPTAuMCwgZGVzY3JpcHRpb249dSdmb3J3YXJkX3ZlbCcsIG1heD0xMC4wLCBtaW49LTEwLjAsIHN0ZXA9Mi4wKSwgRmxvYXTigKY=


linear: 
  x: 0.0
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 0.0
linear: 
  x: 2.0
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 0.0
linear: 
  x: 4.0
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 0.0
linear: 
  x: 2.0
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 0.0
linear: 
  x: 0.0
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 0.0
linear: 
  x: -2.0
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 0.0
linear: 
  x: -4.0
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 0.0
linear: 
  x: -6.0
  y: 0.0
  z: 0.0
angular: 
  x: 0.0
  y: 0.0
  z: 0.0


In [127]:
new_subscriber.unregister()

### ROS  Service
 

Ein **Service** ist eine Funktion, die von einer Node angeboten wird. Sie besteht immer aus Argumenten, die der Node übergeben werden und Resultaten, die das Ergebnis sind. 
Andere Nodes können über diese Services unregelmäßige Ereignisse hervorrufen oder Berechnungen ausführen  lassen.

So kann man sich Services anzeigen lassen

In [139]:
!rosservice list

/Meine_Lieblingsnode_20021_1548403545760/get_loggers
/Meine_Lieblingsnode_20021_1548403545760/set_logger_level
/clear
/kill
/reset
/rosout/get_loggers
/rosout/set_logger_level
/spawn
/turtle1/set_pen
/turtle1/teleport_absolute
/turtle1/teleport_relative
/turtlesim_node/get_loggers
/turtlesim_node/set_logger_level


Wichtige Infos bekommt man über so. Es gibt hier auch ein Format (Type) und Argumente (Args)

In [140]:
!rosservice info /spawn

Node: /turtlesim_node
URI: rosrpc://AURORA-ML:51863
Type: turtlesim/Spawn
Args: x y theta name


In [143]:
!rosservice info /reset

Node: /turtlesim_node
URI: rosrpc://AURORA-ML:51863
Type: std_srvs/Empty
Args: 


Argumente und Rückgabewerte der Services kann man sich so anzeigen lassen:

In [144]:
!rossrv show turtlesim/Spawn

float32 x
float32 y
float32 theta
string name
---
string name



In [146]:
!rossrv show turtlesim/Kill

string name
---



In [129]:
!rossrv show turtlesim/TeleportAbsolute

float32 x
float32 y
float32 theta
---



Services haben Rückmeldungen: mit ServiceProxy kann man ähnlich wie ein Subscriber eine Möglichkeit erzeugen, direkt Services anzusprechen und wie eine normale Funktion zu nutzen.

Wir können einfach einmal eine neue Schildkröte erzeugen. Dabei müssen wir Lage, Orientierung und Name der neuen Kröte angeben:

In [19]:
from turtlesim.srv import Spawn, Kill
from math import pi


## here create a service proxy function for /turtle1/teleport_absolute of TeleportAbsolute type
srv_spawn = rospy.ServiceProxy("/spawn",Spawn)


Return_value = srv_spawn(10,10,0,"NeuerTurtleName")
print("Neuer Name ist " + Return_value.name)


Neuer Name ist NeuerTurtleName


Unsere Schildkröten sind eher Einzelgänger, deswegen sollten wir den neuen Eindringling am besten gleich entfernen ;). Nutze dazu den Service */kill*. Das geht ganz analog zu Spawn!

In [20]:
#implement here!

Man kann seine Kröten auch herumteleportieren!
Dazu kann man den Service */turtle1/teleport_absolute*, analog zu den vorherigen Übungen und teleportiere die Schildkröte mal irgendwo hin ;-)

In [169]:
#implement here!

Jetzt noch schnell das Fenster schließen und den *roscore* beenden!

In [170]:
roscore_process.kill()

## Jetzt können wir das ganze in einem komplexeren Beispiel ausprobieren! 
## [Übung 2](UR5_Sim.ipynb)